In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5, ProgressBar
import numpy as np
import time
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import analysis.switching as sw

In [3]:
from experiments.Pulse_Switching_CSHE_BER import BERExperiment

In [4]:
exp = BERExperiment()
exp.sample = "CSHE2 - C3R3"
exp.comment = "Bit Error Rate - P to AP - 5ns"
exp.polarity = 1 # -1: AP to P; 1: P to AP
exp.field.value = -0.012
exp.pulse_duration.value = 5e-9 # Fixed
exp.reset_amplitude = 0.6
exp.reset_duration  = 5.0e-9
exp.init_instruments()

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die2-C3R3\CSHE2-C3R3-P2AP_2016-06-30_BER_5ns.h5")
# pbar = ProgressBar(num=1, notebook=True)
# edges = [(exp.daq_buffer, wr.data), (exp.daq_buffer, pbar.data)]
edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)

In [6]:
attempts_list = [1 << int(x) for x in np.linspace(14,20,7)]
voltages_list = np.linspace(0.65,0.95,7)
t1 = [] # Keep track of time
t2 = []
for att, vol in tqdm_notebook(zip(attempts_list, voltages_list), total=len(voltages_list)):
    print("Now at ({},{}).".format(att,vol))
    t1.append(time.time())
    exp.pulse_voltage.value = vol
    exp.attempts.value = att
    exp.init_streams()
    exp.reset()
    exp.run_loop()
    t2.append(time.time())
    print("Elapsed time: {} min.".format((t2[-1]-t1[-1])/60))

Now at (512,0.3).
Elapsed time: 0.02768152952194214 min.
Now at (1024,0.5).
Elapsed time: 0.03018232583999634 min.



In [ ]:
print("Total elapsed time: {} min.".format((t2[-1]-t1[0])/60))

In [7]:
# Plot data
data_mean = sw.load_BER_data(wr.filename)
fig = sw.plot_BER(voltages_list, data_mean)
plt.show()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)
